In [234]:
import pandas as pd
import requests
import json
import time
import numpy as np
import os

cwd = os.getcwd() # save current working directory

## Location data

In [231]:
# store llist of kommune codes in list
kom_koder_raw = pd.read_excel(cwd+'/data_download/kom_koder.xls', usecols = 'G:H')
kom_koder = list(kom_koder_raw.iloc[3:, 0])

### Set up functions for school location data

In [221]:
def get_school_data(kom_kode):
    """
    Fetches data for schools by given kommune code and returns a json response.
    """
    url = f'https://dingeologi.appspot.com/_ah/api/skoledistriktendpoint/v1/getSkoler?kommunenr={kom_kode}'
    response = requests.get(url)
    return response.json()


In [222]:
def process_school_data(school_json):
    """
    Takes a json response from get_school_data() as input and returns a dataframe.
    """
    # set up dataframe
    number_of_schools = len(school_json['items'])
    columns = ['navn', 'urlfriendly', 'leder', 'adresse', 'postnr', 'postnrby', 'telefon', 'email', 'www', 'introtekst', 'cvrnr', 'beliggenhedskommune', 'beliggenhedskommunenr', 'administrativkommune', 'opdateret', 'lat', 'lon', 'institutionstype2', 'institutionstype3', 'maxklassetrin', 'ejerforhold', 'oprettet', 'elevtal1516', 'elevtal1415', 'elevtal1314', 'elevtal1213', 'elevtal1112', 'insertdate', 'image', 'active', 'institutionsnummer']
    schools_df = pd.DataFrame(data = None, columns = columns, index = range(number_of_schools))

    # add data
    for i in range(number_of_schools):
        active_school = school_json['items'][i]

        for key, val in active_school.items():
            schools_df.loc[i][key] = val

    schools_df.institutionsnummer = schools_df.institutionsnummer.astype(str)

    return schools_df

### Fetch from API

In [223]:
# MAIN DOWNLOADING PROCESS
# Get json data, process to df and append to list
school_data_list = []

for kom in kom_koder: 
    raw_json = get_school_data(kom)
    school_data_tidy = process_school_data(raw_json)
    school_data_list.append(school_data_tidy)

    time.sleep(0.5)

# Format and save as csv
cols = ['navn', 'postnr', 'beliggenhedskommune', 'beliggenhedskommunenr', 'lat', 'lon', 'institutionstype2', 'institutionstype3', 'institutionsnummer']
all_schools = pd.concat(school_data_list)\
                .reset_index(drop = True)\
                .loc[:,cols]

all_schools.to_csv('school_location.csv')

## Student count and school grades
### Set up functions for formatting

In [225]:
def format_student_count(student_count_raw):
    """
    Formats student count data for merge onto grade and location data. Returns formattedd dataframe.
    """

    student_count = student_count_raw.iloc[:,2:]\
                            .rename(columns = {'Institutionsnummer': 'institutionsnummer'})

    student_count = student_count[student_count['institutionsnummer'] != 'nan']
    student_count.columns = student_count.columns.str.replace('/', '-')

    return student_count



In [226]:
def add_inst_list(raw_df):
    """
    The raw data from excel contains a columns with both school 
    names and institution numbers. This function deletes this
    column and adds a column with only the institution number.

    """

    raw_df['is_inst_num'] = [val.isdecimal() for val in raw_df['Rækkenavne']]

    schools_and_inst = list(raw_df['Rækkenavne'])
    inst_count_dict = {}

    for i in range(len(schools_and_inst)):
        if schools_and_inst[i].isdecimal():
            active_num = schools_and_inst[i]
            count = 0
        else:
            count += 1
            inst_count_dict[active_num] = count
    
    inst_nested = [[num]*count for num, count in inst_count_dict.items()]
    inst_list = [num for inst_list in inst_nested for num in inst_list]

    df_out = raw_df[raw_df.is_inst_num == False]\
             .drop(columns = 'is_inst_num')
    df_out['institutionsnummer'] = inst_list

    return df_out

## Merge grades, location and student count

In [227]:
def get_all_data(end_year, grades_raw, student_count_raw, school_loc_raw):
    """
    This function loads raw grade data and formatted student count data for the
    specified year as well as dirty school location data and merges into one
    combined dataframe ready for datacleaning.
    """
    year = str(end_year-1)+'-'+str(end_year)
    grades = add_inst_list(grades_raw)
    student_count = format_student_count(student_count_raw) 

    # prepare format for merge
    for df in [grades, student_count, school_loc_raw]:
        df.institutionsnummer = df.institutionsnummer.astype(str)

    # merge graeds to student count
    school_grade_count = pd.merge(left = grades, right = student_count[['institutionsnummer', year]], how = 'left', on = 'institutionsnummer')\
                            .rename(columns = {year: 'student_count'})
    # merge to location data
    school_grade_count_loc = pd.merge(left = school_grade_count, right = school_loc_raw, on = 'institutionsnummer', how = 'outer')

    return school_grade_count_loc

## Clean data

In [228]:
def clean_school_data(dirty_df):
    """
    Extracts overall grade point average (true value as well as the socioeconomic reference) for
    schools in input dataframe. Reshapes dataframe to one school/location point pr. row. Saves
    grade data, location data, student count as well as type of institution for further sorting.

    """

    avg_df = dirty_df[dirty_df['Rækkenavne'] == 'Gennemsnit'] # might update later to extract grades                                                                   for all subjects
    # Define variables of interest, split into text and numeric
    num_vars = ['Karakter', 'Soc_ref', 'student_count', 'postnr', 'beliggenhedskommunenr', 'lat', 'lon']
    text_vars = ['navn', 'institutionstype2', 'institutionstype3']
    
    # Extract and clean text variables
    temp_text = avg_df.pivot(columns = 'institutionsnummer', values = text_vars)\
                        .transpose()\
                        .stack()\
                        .unstack(level = 0)\
                        .reset_index(col_level = -1)
    temp_text.columns = ['institutionsnummer', 'level_1', 'navn', 'institutionstype2', 'institutionstype3']
    text_df = temp_text.drop_duplicates('institutionsnummer')\
                        .drop('level_1', axis = 1)
    
    # Extract and clean numeric variables
    num_df = pd.pivot_table(data = avg_df, index = 'institutionsnummer', values = num_vars)\
                .reset_index()
    
    
    # Merge together and drop missing values
    # OBS: for missing values we have only grade data, no location data.
    tidy = pd.merge(left = num_df, right = text_df, on = 'institutionsnummer', how = 'left')\
            .dropna()

    # Add geo coordinates and format for merge
    tidy['coordinates'] = tuple(zip(tidy['lat'], tidy['lon']))
    tidy.postnr = tidy.postnr.astype(int)
    tidy.beliggenhedskommunenr = tidy.beliggenhedskommunenr.astype(int)

    return tidy



In [229]:
# MAIN DATA CLEANING PROCESS
# Get data for school location, grades and student count, merge, clean and save to csv
years = ['2009-2010', '2010-2011', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018-2019']

schools_clean = {} # save as dict if further processing is needed

student_count = pd.read_excel(cwd+'/data_download/student_count.xls', header = 7)
school_location = pd.read_csv('school_location_dirty.csv')

for year in years:
    grades = pd.read_excel(cwd+f'/data_download/grades_{year}.xls', header = 6)
    end_year = int(year[-4:])

    dirty = get_all_data(end_year, grades, student_count, school_location)
    clean = clean_school_data(dirty)

    schools_clean[year] = clean
    clean.to_csv(f'school_clean_{year}.csv')